# Financial analysis

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `dataproject.py` file includes a function which will be used multiple times in this notebook.

Import packages and data file:

In [35]:
import pandas as pd
import statsmodels.formula.api as sm
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.stats.outliers_influence as sm_influence
from patsy import dmatrices
import statsmodels.api as sm_non_formula

# Read data

**Import data .CSV file**

In [36]:
financials = pd.read_csv("/Users/williamdybdahl/Documents/Polit/Cand_3/Programming & Numerical Analysis/Financials.csv")

Print 20 first rows to get overview of data structure

In [37]:
print(financials.head(20))

    gvkey  datadate   tic        AP         AT        BV       COGS      EMP  \
0    1004  20130531   AIR   149.300   2136.900   918.600   1714.500    6.300   
1    1004  20140531   AIR   171.100   2199.500   999.500   1581.400    5.800   
2    1004  20150531   AIR   142.300   1515.000   845.100   1342.700    4.850   
3    1004  20160531   AIR   163.400   1442.100   865.800   1354.900    4.700   
4    1004  20170531   AIR   177.400   1504.100   914.200   1422.700    4.600   
5    1045  20121231   AAL  1244.000  23510.000 -7987.000  20529.000   77.750   
6    1045  20131231   AAL  1368.000  42278.000 -2731.000  19084.000  110.400   
7    1045  20141231   AAL  1377.000  43771.000  2021.000  29511.000  113.300   
8    1045  20151231   AAL  1563.000  48415.000  5635.000  25416.000  118.500   
9    1045  20161231   AAL  1592.000  51274.000  3785.000  25695.000  122.300   
10   1050  20121231  CECE    11.098     94.104    61.994     91.690    0.452   
11   1050  20131231  CECE    23.108    3

# Investigate and clean data

Identify missing values/null values. Sum over both columns and rows (axis 0 and 1)

In [38]:
print(financials.isnull().sum().head(20))
print(financials.isnull().sum(axis=1).head(20))

gvkey              0
datadate           0
tic               38
AP             12693
AT              9482
BV             14858
COGS           14531
EMP            13105
IBE             9607
OCF            14702
PPE            17100
REC            14634
SALE            9618
XOPR           14536
MV             17492
2_digit_sic        0
dtype: int64
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    3
16    7
17    7
18    3
19    7
dtype: int64


Create new column in dataframe containing sum of missing values, and sort financial data descending after new NumNulls column

In [39]:
financials['NumNull']=financials.isnull().sum(axis=1)
financials = financials.sort_values('NumNull', ascending=False)
print(financials.head(20))

        gvkey  datadate  tic  AP  AT  BV  COGS  EMP  IBE  OCF  PPE  REC  SALE  \
36540  177391  20141231  NaN NaN NaN NaN   NaN  NaN  NaN  NaN  NaN  NaN   NaN   
35569  176324  20141231  NaN NaN NaN NaN   NaN  NaN  NaN  NaN  NaN  NaN   NaN   
35567  176324  20121231  NaN NaN NaN NaN   NaN  NaN  NaN  NaN  NaN  NaN   NaN   
37047  178034  20141231  NaN NaN NaN NaN   NaN  NaN  NaN  NaN  NaN  NaN   NaN   
37045  178034  20121231  NaN NaN NaN NaN   NaN  NaN  NaN  NaN  NaN  NaN   NaN   
36695  177503  20131231  NaN NaN NaN NaN   NaN  NaN  NaN  NaN  NaN  NaN   NaN   
36696  177503  20141231  NaN NaN NaN NaN   NaN  NaN  NaN  NaN  NaN  NaN   NaN   
35451  176211  20131231  NaN NaN NaN NaN   NaN  NaN  NaN  NaN  NaN  NaN   NaN   
35452  176211  20141231  NaN NaN NaN NaN   NaN  NaN  NaN  NaN  NaN  NaN   NaN   
35453  176212  20121231  NaN NaN NaN NaN   NaN  NaN  NaN  NaN  NaN  NaN   NaN   
35454  176212  20131231  NaN NaN NaN NaN   NaN  NaN  NaN  NaN  NaN  NaN   NaN   
35455  176212  20141231  NaN

We drop IBE, OCF and PPE as they seem irrelevant and contain a significant number of missing values.
Also drop all rows with more than 3 missing values. 

In [40]:
financials = financials.drop(['IBE', 'OCF', 'PPE'], axis=1)
financials.dropna(thresh=13, inplace=True)
financials.dropna(subset=['SALE'], inplace=True)
print(financials.head(20))
print(financials.isnull().sum())

        gvkey  datadate    tic        AP         AT        BV      COGS  \
25116  107629  20161231  3NODB  1218.840   1367.985   140.580     2.386   
26420  119414  20170930   CFFN  5309.868   9192.916  1368.313   117.804   
9975    18035  20161231   EXSR  1943.410   2179.401   189.172     0.249   
8835    16957  20140930   HARL   502.251    791.353    63.162    11.041   
9974    18035  20151231   EXSR  1838.977   2062.508   173.107    -1.429   
9972    18035  20141231   EXSR  1668.399   1887.679   160.074     2.511   
14820   23060  20161231   BHWB   572.366    665.728    49.550     4.521   
26605  121073  20141231   AMBZ  1364.444   1535.538   118.891     2.430   
26846  122954  20121231  9769B   118.488  17937.257  6136.828  5968.469   
26607  121073  20151231   AMBZ  1524.136   1671.257   131.383     1.501   
25114  107629  20151231  3NODB  1205.246   1352.286   136.524     3.219   
9910    17987  20121231   ESBK   409.565    536.936    42.795     6.359   
9135    17222  20121231  

From tables above, we see that data is already more clean. Number of variables with significant number of missing values are lower. For variables AP, REC and BV we fill out missing values with total column means.

In [42]:
financials.AP.fillna(financials.AP.mean(), inplace=True)
financials.REC.fillna(financials.REC.mean(), inplace=True)
financials.BV.fillna(financials.BV.mean(), inplace=True)

print(financials.isnull().sum())

gvkey             0
datadate          0
tic               0
AP                0
AT                0
BV                0
COGS              0
EMP            1371
REC               0
SALE              0
XOPR              0
MV             4873
2_digit_sic       0
NumNull           0
dtype: int64


At last we replace missing MV and EMP values with industry means (based on sic code)

In [44]:
financials.MV.fillna(financials.groupby('2_digit_sic')['MV'].transform("mean"), inplace=True)
financials.EMP.fillna(financials.groupby('2_digit_sic')['EMP'].transform("mean"), inplace=True)
print(financials.isnull().sum())

gvkey          0
datadate       0
tic            0
AP             0
AT             0
BV             0
COGS           0
EMP            0
REC            0
SALE           0
XOPR           0
MV             0
2_digit_sic    0
NumNull        0
dtype: int64


We have now cleaned data such that **there are no missing values**. We print descriptive statistics to get an overview of each variable.

In [45]:
print(financials.describe())

               gvkey      datadate            AP            AT             BV  \
count   28679.000000  2.867900e+04  2.867900e+04  2.867900e+04   28679.000000   
mean    82894.917501  2.014338e+07  4.553832e+03  1.625748e+04    2711.770446   
std     76067.848835  1.416421e+04  5.376664e+04  1.216392e+05   12544.056063   
min      1004.000000  2.012113e+07  0.000000e+00  0.000000e+00 -132599.000000   
25%     17131.500000  2.013123e+07  2.183000e+00  6.380450e+01      17.040000   
50%     60972.000000  2.014123e+07  2.729000e+01  6.296590e+02     179.086000   
75%    162528.000000  2.015123e+07  2.822010e+02  3.632374e+03    1082.396000   
max    318815.000000  2.017093e+07  1.709017e+06  3.287968e+06  284434.000000   

                COGS           EMP           REC           SALE  \
count   28679.000000  28679.000000  2.867900e+04   28679.000000   
mean     2854.858348     11.565645  5.344330e+03    4284.453183   
std     14384.580698     47.652655  6.449719e+04   18677.014588   
mi

From the descriptive statistics we see that a lot of the variables contain negative values. For a number of variables this does not make any sense, thus we clean out negative values in XOPR, COGS, SALE, BV and EMP.

In [46]:
financials = financials[financials.XOPR > 0]
financials = financials[financials.COGS > 0]
financials = financials[financials.SALE > 0]
financials = financials[financials.BV > 0]
financials = financials[financials.EMP > 0]

print(financials.describe())

               gvkey      datadate            AP            AT             BV  \
count   22717.000000  2.271700e+04  2.271700e+04  2.271700e+04   22717.000000   
mean    79820.421843  2.014341e+07  5.718313e+03  1.904609e+04    3466.732796   
std     75428.677692  1.422304e+04  6.035161e+04  1.243067e+05   13778.660310   
min      1004.000000  2.012113e+07  0.000000e+00  2.000000e-03       0.002000   
25%     15334.000000  2.013123e+07  5.950000e+00  1.830270e+02      66.202000   
50%     31358.000000  2.014123e+07  5.820000e+01  1.016494e+03     329.258000   
75%    158053.000000  2.015123e+07  4.432000e+02  4.996427e+03    1543.780000   
max    318815.000000  2.017093e+07  1.709017e+06  2.692538e+06  284434.000000   

                COGS           EMP           REC           SALE  \
count   22717.000000  22717.000000  2.271700e+04   22717.000000   
mean     3468.468616     13.678329  5.634741e+03    5186.278388   
std     16014.760246     52.222003  5.023828e+04   20696.732755   
mi

We have now cleaned data such that there are no negative values where those do not make sense. Since we are going to use the data for OLS regresionnal analysis, we are also interested in varibales with significant degree of skewness. To handle this for COGS, we normalize by dividing difference to mean with max difference. This yields values only between 0 and 1

In [48]:
financials.COGS = (financials.COGS - financials.COGS.min())/(financials.COGS.max() - financials.COGS.min())

print(financials.describe())

               gvkey      datadate            AP            AT             BV  \
count   22717.000000  2.271700e+04  2.271700e+04  2.271700e+04   22717.000000   
mean    79820.421843  2.014341e+07  5.718313e+03  1.904609e+04    3466.732796   
std     75428.677692  1.422304e+04  6.035161e+04  1.243067e+05   13778.660310   
min      1004.000000  2.012113e+07  0.000000e+00  2.000000e-03       0.002000   
25%     15334.000000  2.013123e+07  5.950000e+00  1.830270e+02      66.202000   
50%     31358.000000  2.014123e+07  5.820000e+01  1.016494e+03     329.258000   
75%    158053.000000  2.015123e+07  4.432000e+02  4.996427e+03    1543.780000   
max    318815.000000  2.017093e+07  1.709017e+06  2.692538e+06  284434.000000   

               COGS           EMP           REC           SALE           XOPR  \
count  22717.000000  22717.000000  2.271700e+04   22717.000000   22717.000000   
mean       0.007960     13.678329  5.634741e+03    5186.278388    4196.117019   
std        0.036754     52.

In [49]:
#financials['Binned_SALE']=pd.qcut(financials.SALE, 10, labels=False)
#print(financials.describe())